# Imports

In [1]:
import numpy as np
import os
import json
import pickle
import tarfile
import datetime
import cv2
from skimage.feature import hog
from skimage.feature import graycomatrix, graycoprops
#from skimage.feature import greycomatrix, greycoprops
from skimage import data, exposure
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import warnings
from utils import *

In [2]:
warnings.filterwarnings("ignore")

#Global knobs
debug = False

# Number of KMeans clusters to create for SIFT Bag of Visual Words
NUM_KMEANS_CLUSTER = 20

# Set this to zero, if all images in train/val/test sets have to be examined
# For shorter test runs, set this to a reasonable non-zero value (like 100) for
# quick functional verification
LIMIT_NUM_IMAGES = 0

# Define Helper Functions

In [3]:
# define a function that determines the mean and standard deviation of each RGB
# color-space channel for an image
def compute_channel_stats(image_path):
    # read the image
    img = cv2.imread(image_path)
    
    # compute mean and standard deviation for each color channel (RGB)
    mean_rgb, std_rgb = cv2.meanStdDev(img)
    
    # flatten the results into a feature vector
    channel_stats = np.concatenate((mean_rgb.flatten(), std_rgb.flatten()))
    
    return channel_stats

In [4]:
# define a function to generate a grid of smoothed distributions of mean intensity counts in each channel
# for each class across all images in each class
def compute_channel_distributions(image_path, bins=20, channels='hsv'):
    
    if channels=='rgb':
        ch1 = 'r' #'Red'
        ch2 =  'g' #'Green'
        ch3 = 'b' #'Blue'
    elif channels=='lab':
        ch1 = 'L*'
        ch2 = 'a*'
        ch3 = 'b*'
    elif channels=='hsv':
        ch1 = 'h' #'Hue'
        ch2 = 's' #'Saturation'
        ch3 = 'v' #'Value'

    # Load the image
    img = plt.imread(image_path)
    if channels=='lab':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB) # convert image to L*a*b* color space
    elif channels=='hsv':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # convert image to HSV color space
    
    # calculate histograms for each color channel
    ch1_hist, _ = np.histogram(img[:,:,0], bins=bins, range=(0, 255))
    ch2_hist, _ = np.histogram(img[:,:,1], bins=bins, range=(0, 255))
    ch3_hist, _ = np.histogram(img[:,:,2], bins=bins, range=(0, 255))
      
    # generate a vector that concatenates all 3 channel distributions
    ch_distributions = np.concatenate((ch1_hist, ch2_hist, ch3_hist))
    
    # generate a list of feature names
    feature_names = [f"{ch1}_{i}" for i in range(1,bins+1)] + [f"{ch2}_{i}" for i in range(1,bins+1)] + [f"{ch3}_{i}" for i in range(1,bins+1)]
        

    return ch_distributions, feature_names

In [5]:
# define a function that determines the hog descriptors for an image's grayscale representation
def compute_hog_stats(image_path):
    # read the image
    img = cv2.imread(image_path)
    
    # convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # compute HOG features
    fd = hog(gray_img, orientations=4, pixels_per_cell=(32, 32), feature_vector=True)
    
    return fd

In [6]:
def compute_glcms(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  
    
    # distance between pixels
    distances = [1]  
    # angles for texture computation
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4] 
    glcm = graycomatrix(img, distances, angles, symmetric=True, normed=True)
    
    contrast = graycoprops(glcm, 'contrast').ravel().mean()
    dissimilarity = graycoprops(glcm, 'dissimilarity').ravel().mean()
    homogeneity = graycoprops(glcm, 'homogeneity').ravel().mean()
    energy = graycoprops(glcm, 'energy').ravel().mean()
    correlation = graycoprops(glcm, 'correlation').ravel().mean()
    
    return [contrast, dissimilarity, homogeneity, energy, correlation]

In [7]:
def compute_fft(image_path, filt):
    FREQBINS = 25

    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  
    filtered_image = apply_gaussian_filter(img, filt)
    magnitude_spectrum = fft_image(filtered_image)
    spec = np.log(1+magnitude_spectrum).ravel()
            
    hist, bins = np.histogram(spec, bins=FREQBINS)
    
    # We don't particularly care about the exact bin boundaries. 
    # Just need the distribution of freq spectrum.

    return hist

In [8]:
def extract_key_points(img):

    # Converting image to grayscale
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Applying SIFT detector
    sift = cv2.SIFT_create(nfeatures=0, nOctaveLayers=3, edgeThreshold=0.2, contrastThreshold=0.07)

    #kp = sift.detect(gray, None)
    kp, des = sift.detectAndCompute(gray,None)

    # Marking the keypoint on the image using circles
    sift_img=cv2.drawKeypoints(gray, kp, img,
                          flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    return sift_img, des

In [9]:
# Function to extract SIFT keypoint descriptors 
def generate_sift_vectors (files, directory):

    # create a list to store SIFT Keypoint descriptors
    sift_vectors = []
    
    test_num = 0
    
    # iterate over each file
    for class_name, file_name in files:
        
        # load the image
        img_path = os.path.join(directory, class_name, file_name)
        img = plt.imread(img_path)

        # Extract SIFT keypoints
        sift_image, kp_descriptors = extract_key_points(img)

        sift_vectors.append((kp_descriptors, class_name))
        
        test_num += 1     
        if LIMIT_NUM_IMAGES > 0 and test_num > LIMIT_NUM_IMAGES:
            break

    return sift_vectors

In [10]:
def generate_sift_BoW_hist(files, directory):
    feature_vectors = {}
    
    sift_vectors = generate_sift_vectors(files, directory)
    
    # Stack all the SIFT kp descriptors of all images vertically 
    # so that we get one giant 2D array: 
    # Num keypoints across all images x size of each descriptor (128)
    vector_kps, vector_class = sift_vectors[0]
    vStack = np.array(vector_kps)
    for remaining in sift_vectors[1:]:
        vector_kps, vector_class = remaining
        if vector_kps is not None:
            vStack = np.vstack((vStack, vector_kps))
    if debug:
        print(vStack.shape)
    
    # Form clusters of visual words using KMeans
    kmeans = KMeans(init="k-means++", n_clusters=NUM_KMEANS_CLUSTER, n_init=4)
    kmeans_fit = kmeans.fit_predict(vStack)

    if debug:
        print(f'Number of KPs: {kmeans_fit.shape}, MinCluster: {kmeans_fit.min()}, MaxCluster: {kmeans_fit.max()}')

    # Create a histogram for each image - each kp for the image belongs to one 
    # bucket (visual word). Keep a count of such visual words per image.
    num_images = len(sift_vectors)
    histogram = np.array([np.zeros(NUM_KMEANS_CLUSTER) for i in range(num_images)])
    count = 0
    for img_num in range(num_images):
        vector_kps, vector_class = sift_vectors[img_num]
        if vector_kps is None:
            # Some images have zero keypoints. 
            num_kp_in_image = 0
        else:
            num_kp_in_image = len(vector_kps)
        for j in range(num_kp_in_image):
            idx = kmeans_fit[count+j]
            histogram[img_num][idx] += 1
        count += num_kp_in_image

        # append each combined_features array to the correct class in feature_vectors
        if vector_class not in feature_vectors:
            feature_vectors[vector_class] = []
        feature_vectors[vector_class].append(histogram[img_num])
        
    sift_BoW_names = [f"sift_bow_{i}" for i in range(NUM_KMEANS_CLUSTER)]
    
    return feature_vectors, sift_BoW_names


In [11]:
# define a function that loops through each file to generate a dictionary that contains
# the feature vectors of all images in each class
def generate_feature_vectors(files, directory):
    feature_vectors = {}
    image_list = {}
    test_num = 0

    # The 256x256 was based on EDA on the image size across the various classes.
    # The window size 25 was established by experimentation on image blurring.
    filt = create_gaussian_filter(256, 256, 25)
    
    # iterate over each file
    for class_name, file_name in files:
        # load the image
        img_path = os.path.join(directory, class_name, file_name)

        # compute color statistics
        channel_stats = compute_channel_stats(img_path)
        
        # compute channel distributions
        channel_distributions, ch_dist_names = compute_channel_distributions(img_path)
        
        # compute HOG features
        hog_stats = compute_hog_stats(img_path)
        
        # compute GLCM texture features
        glcm_features = compute_glcms(img_path)
        
        # compute Freq spectrum features
        freq_features = compute_fft(img_path, filt)
    
        combined_features = np.concatenate((channel_stats, channel_distributions, hog_stats, glcm_features, freq_features))
        
        # append each combined_features array to the correct class in feature_vectors
        if class_name not in feature_vectors:
            feature_vectors[class_name] = []
        feature_vectors[class_name].append(combined_features)
        
        if class_name not in image_list:
            image_list[class_name] = []
        image_list[class_name].append(img_path)
        
        test_num += 1
        if LIMIT_NUM_IMAGES > 0 and test_num >= LIMIT_NUM_IMAGES:
            break
        
    hog_feature_names = [f"hog_{i}" for i in range(hog_stats.shape[0])]
    frq_feature_names = [f"freqbin_{i}" for i in range(freq_features.shape[0])]
    
    return feature_vectors, image_list, ch_dist_names, hog_feature_names, frq_feature_names

In [12]:
# define a function to save the feature vector dictionary to disk
def save_feature_data(feature_vectors, images_in_class, feature_names, file_directory, file_desc):
    
    # save vectors
    vectors_filename = os.path.join(file_directory, 'feature_vectors_' + file_desc + '.tar.gz')
   
    # convert numpy arrays to Python lists
    feature_vectors_dict = {}
    for class_name, vectors in feature_vectors.items():
        feature_vectors_dict[class_name] = [vector.tolist() for vector in vectors]

    # save feature_vectors_dict dictionary as JSON
    json_filename = vectors_filename.replace('.tar.gz', '.json')
    with open(json_filename, 'w') as f:
        json.dump(feature_vectors_dict, f)
    
    if debug:
        display_counts_per_class(json_filename, feature_vectors_dict)
    
    # create tar.gz file
    with tarfile.open(vectors_filename, 'w:gz') as tar:
        tar.add(json_filename, arcname=os.path.basename(json_filename))
    
    # remove the temporary JSON file
    os.remove(json_filename)

    # save image list
    images_filename = os.path.join(file_directory, 'image_list_' + file_desc + '.tar.gz')
    
    # save feature_vectors_dict dictionary as JSON
    json_filename = images_filename.replace('.tar.gz', '.json')
    with open(json_filename, 'w') as f:
        json.dump(images_in_class, f)

    if debug:
        display_counts_per_class(json_filename, images_in_class)

    # create tar.gz file
    with tarfile.open(images_filename, 'w:gz') as tar:
        tar.add(json_filename, arcname=os.path.basename(json_filename))
    
    # remove the temporary JSON file
    os.remove(json_filename)
    
    # save names
    names_filename = os.path.join(file_directory, 'feature_names.pkl')
    with open(names_filename, 'wb') as f:
        pickle.dump(feature_names, f)

# Load data

In [13]:
# define file directory
directory = '../data/interim/PatternNet/PatternNet/images'

# create a list of classes considered for this project
classes = ['beach', 'chaparral', 'dense_residential', 'forest', 'freeway', 'harbor', 'overpass', 'parking_space', 'river', 'swimming_pool']

# define the train, val, and test sets
train_files, val_files, test_files = generate_splits(classes, directory)

train/validation/test subsets were loaded from a pre-generated file
	Number of train files: 4799
	Number of val files: 1599
	Number of test files: 1601


In [14]:
def generate_consolidated_feature_vectors(files, directory):
    start_time = datetime.datetime.now()

    # generate the set of feature vectors for all images in each class of the training set
    feature_vectors, image_list, ch_dist_names, hog_feature_names, frq_feature_names = generate_feature_vectors(files, directory)

    # generate BoVW histogram using SIFT features;
    # Add them to the feature_vectors_train, so that the rest of the 
    # processign is common
    sift_feature_vectors, sift_feature_names = generate_sift_BoW_hist(files, directory)

    # Concatenate feature_vectors_train and sift_feature_vectors
    for cls in feature_vectors.keys():
        for idx,val in enumerate(feature_vectors[cls]):
            feature_vectors[cls][idx] = np.hstack((feature_vectors[cls][idx], \
                                    sift_feature_vectors[cls][idx]))

    end_time = datetime.datetime.now()
    elapsed_time = end_time - start_time
    print("Time taken for training:", elapsed_time)

    return feature_vectors, image_list, ch_dist_names, hog_feature_names, frq_feature_names, sift_feature_names


# Generate feature data

In [15]:
# Invoke wrapper function to combine all features including SIFT

feature_vectors_train, image_list_train, ch_dist_names, hog_feature_names, frq_feature_names, sift_feature_names  = generate_consolidated_feature_vectors(train_files, directory)

Time taken for training: 0:28:06.761285


In [16]:
# inspections

print(type(feature_vectors_train))
print(feature_vectors_train.keys())
print(feature_vectors_train['beach'][0].shape)
print(len(feature_vectors_train['beach']))
print(len(feature_vectors_train['beach'][0]))
print(feature_vectors_train['beach'][0])


<class 'dict'>
dict_keys(['parking_space', 'beach', 'forest', 'overpass', 'river', 'dense_residential', 'swimming_pool', 'chaparral', 'freeway', 'harbor'])
(1412,)
479
1412
[113.95576477 127.46353149 119.84751892 ...  22.           1.
   5.        ]


In [17]:
# create a list of feature names
rgb_names = ['r_mean','g_mean','b_mean','r_std','g_std','b_std'] # 6
hsv_names = ch_dist_names # 60
hog_names = hog_feature_names # 1296
texture_names = ['contrast_mean','dissimilarity_mean','homogeneity_mean','energy_mean','correlation_mean'] # 5
frq_names = frq_feature_names # 25
sift_names = sift_feature_names
feature_names = rgb_names + hsv_names + hog_names + texture_names + frq_names + sift_names
print(feature_names)

['r_mean', 'g_mean', 'b_mean', 'r_std', 'g_std', 'b_std', 'h_1', 'h_2', 'h_3', 'h_4', 'h_5', 'h_6', 'h_7', 'h_8', 'h_9', 'h_10', 'h_11', 'h_12', 'h_13', 'h_14', 'h_15', 'h_16', 'h_17', 'h_18', 'h_19', 'h_20', 's_1', 's_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13', 's_14', 's_15', 's_16', 's_17', 's_18', 's_19', 's_20', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'v_15', 'v_16', 'v_17', 'v_18', 'v_19', 'v_20', 'hog_0', 'hog_1', 'hog_2', 'hog_3', 'hog_4', 'hog_5', 'hog_6', 'hog_7', 'hog_8', 'hog_9', 'hog_10', 'hog_11', 'hog_12', 'hog_13', 'hog_14', 'hog_15', 'hog_16', 'hog_17', 'hog_18', 'hog_19', 'hog_20', 'hog_21', 'hog_22', 'hog_23', 'hog_24', 'hog_25', 'hog_26', 'hog_27', 'hog_28', 'hog_29', 'hog_30', 'hog_31', 'hog_32', 'hog_33', 'hog_34', 'hog_35', 'hog_36', 'hog_37', 'hog_38', 'hog_39', 'hog_40', 'hog_41', 'hog_42', 'hog_43', 'hog_44', 'hog_45', 'hog_46', 'hog_47', 'hog_48', 'hog_49',

In [18]:
# generate the set of feature vectors for all images in each class of the training set
feature_vectors_val, image_list_val, ch_dist_names, hog_feature_names, frq_feature_names, sift_feature_names = generate_consolidated_feature_vectors(val_files, directory)

# generate the set of feature vectors for all images in each class of the training set
feature_vectors_test, image_list_test, ch_dist_names, hog_feature_names, frq_feature_names, sift_feature_names = generate_consolidated_feature_vectors(test_files, directory)

Time taken for training: 0:04:23.960015
Time taken for training: 0:04:35.339901


# Save feature data to disk

In [19]:
# save feature data to disk
save_feature_data(feature_vectors_train, image_list_train, feature_names, "../data/processed/", "train")
save_feature_data(feature_vectors_val, image_list_val, feature_names, "../data/processed/", "val")
save_feature_data(feature_vectors_test, image_list_test, feature_names, "../data/processed/", "test")

In [20]:
len(feature_vectors_val['river'])

160

In [21]:
len(image_list_val['river'])

160